# SEIRX model runs for test technology and screening frequency investigation in schools

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from os.path import join
import os
import shutil
import pickle
import json

import sys
sys.path.insert(0,'../school')
sys.path.insert(0,'../nursing_home')
from model_school import SEIRX_school
import analysis_functions as af

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display
import time

## Model setup

### Screening parameters

In [2]:
## parameter ranges 
# student and teacher streening intervals (in days)
s_screen_range = [None, 3, 7, 14]
t_screen_range = [None, 3, 7, 14]
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen', 'one_day_PCR', 'two_day_PCR']
# specifies whether the index case will be introduced via an
# employee or a resident
index_cases = ['student', 'teacher']
# specifies whether teachers wear masks
masks = [True, False]

params = [(i, j, k, l, m) for i in test_types \
                          for j in index_cases \
                          for k in s_screen_range \
                          for l in t_screen_range \
                          for m in masks]
# currently 192 param combinations
N_configs = len(params)

### Agents

In [3]:
agent_types = {
        'student':{
                'screening_interval': None, # screening param
                'index_probability': 0, # running in single index case mode
                'transmission_risk': 0.01, # calibrated
                'reception_risk': 1, # fixed
                'mask':False}, # screening param
        'teacher':{
                'screening_interval': None, # screening param
                'index_probability': 0, # running in single index case mode
                'transmission_risk': 0.01, # calibrated
                'reception_risk': 1, # fixed
                'mask':False},
        'family_member':{
                'screening_interval': None, # fixed 
                'index_probability': 0, # fixed
                'transmission_risk': 0.01, # calibrated
                'reception_risk': 1, # fixed
                'mask':False} # screening param
}

In [4]:
def sample_prevention_strategies(params, school, agent_types, res_path, runs,\
                                 verbose=False):
    
    ## fixed parameters
    # maximum number of steps in a single run. A run automatically stops if the 
    # outbreak is contained, i.e. there are no more infected or exposed agents.
    N_steps = 500 
    testing = 'preventive'
    half_classes = False
    diagnostic_test_type = 'two_day_PCR'
    
    
    ## data I/O
    # construct folder for results if not yet existing
    school_name = '{}_classes-{}_students-{}_floors-{}'.format(\
        school['type'],school['classes'],school['students'],school['floors'])
    try:
        os.mkdir(join(res_path + '/results', school_name))
    except FileExistsError:
        pass             
    # load the contact network, schedule and node_list corresponding to the school
    try: 
        G = nx.readwrite.gpickle.read_gpickle(join(res_path + '/networks',\
                                            '{}.gpickle'.format(school_name)))
    # if there is no network, the school does not exist. This can happen if for
    # example the number of floors is larger than the number of classes. These
    # school characteristics combinations are ignored
    except FileNotFoundError:
        return
    schedule = pd.read_csv(join(res_path + '/schedules',\
                                '{}_schedule.csv'.format(school_name)))
    schedule.index = schedule['teacher']
    schedule = schedule.drop(columns=['teacher'])
    node_list = pd.read_csv(join(res_path + '/node_lists',\
                                 '{}_node_list.csv'.format(school_name)))
    

    ## scan of all possible parameter combinations of prevention measures
    observables = pd.DataFrame()
    c = 0
    for ttype, index_case, s_screen_interval, t_screen_interval, mask in params:
        agent_types['student']['screening_interval'] = s_screen_interval
        agent_types['teacher']['screening_interval'] = t_screen_interval
        agent_types['teacher']['mask'] = mask
                        
        if verbose and c % 10 == 0:
            print('config {} / {}'.format(c, N_configs))
        c += 1

        # temporary folder for all runs in the ensemble, will be
        # deleted after a representative run is picked
        try:
            shutil.rmtree(join(res_path + '/results', 'tmp'))
        except FileNotFoundError:
            pass
        os.mkdir(join(res_path + '/results', 'tmp'))

        # results of one ensemble with the same parameters
        ensemble_results = pd.DataFrame()
        for r in range(runs):
            # instantiate model with current scenario settings
            model = SEIRX_school(G, 
              testing = testing, # fixed: True
              diagnostic_test_type = diagnostic_test_type, # fixed: 2 day PCR)
              preventive_screening_test_type = ttype, # screen param
              index_case = index_case, # screen param
              agent_types = agent_types, # 3 more screen params
              exposure_duration = [5, 1.9], # literature values
              time_until_symptoms = [6.4, 0.8], # literature values
              infection_duration = [10.91, 3.95], # literature values
              age_transmission_risk_discount = \
                        {'slope':-0.05, 'intercept':1}, # calibrated
              age_symptom_discount = \
                        {'slope':-0.02545, 'intercept':0.854545}) # calibrated

            # run the model, end run if the outbreak is over
            for i in range(N_steps):
                model.step()
                if len([a for a in model.schedule.agents if \
                    (a.exposed == True or a.infectious == True)]) == 0:
                    break

            # collect the statistics of the single run
            row = af.get_ensemble_observables_school(model, r)
            # add run results to the ensemble results
            ensemble_results = ensemble_results.append(row,
                ignore_index=True)

            N_infected = row['infected_agents']

            with open(join(join(res_path + '/results', 'tmp'),\
                'run_{}_N_{}.p'.format(r,N_infected)),'wb') as f:
                pickle.dump(model, f)

       # add ensemble statistics to the overall results
        row = {'test_type':ttype,
               'index_case':index_case,
               'student_screen_interval':s_screen_interval,
               'teacher_screen_interval':t_screen_interval,
               'mask':mask}
        for col in ensemble_results.columns:
            row.update(af.get_statistics(ensemble_results, col))
        observables = observables.append(row, ignore_index=True)

        rep_model = af.get_representative_run(row['infected_agents_median'],\
                            join(res_path + '/results', 'tmp'))
        tm_events = af.get_transmission_chain(rep_model, schedule)
        state_data = af.get_agent_states(rep_model, tm_events)

        af.dump_JSON(join(res_path + '/results', school_name),
                     school,
                     ttype, index_case, s_screen_interval,
                     t_screen_interval, mask, half_classes,
                     node_list, schedule, tm_events, state_data)

    # cleanup & save results to disk
    shutil.rmtree(join(res_path + '/results', 'tmp'))
    observables.to_csv(join(join(res_path + '/results', school_name),\
            'observables_N{}.csv'.format(runs)), index=False)

In [10]:
runs = 10
school = {'type':'unterstufe',
          'classes':4,
          'students':10,
          'floors':1}

res_path = '../data/school' 

class_sizes = [10, 15, 20, 25, 30]
class_numbers = [4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 50, 75, 100]
floor_numbers = [1, 2, 3, 4]
school_types = ['volksschule']

#class_sizes = [10]
#class_numbers = [4]
#floor_numbers = [1]
#school_types = ['volksschule']

school_configs = [(i, j, k, l) for i in school_types \
                               for j in class_numbers \
                               for k in class_sizes \
                               for l in floor_numbers]

f = IntProgress(min=0, max=len(school_configs))
display(f)
c = 0 # counter for progress bar
for school_type, N_classes, class_size, N_floors in school_configs:
    school = {'type':school_type, 'classes':N_classes,
              'students':class_size, 'floors':N_floors}

    f.value = c # update the progress bar
    c += 1

    res_path = '../data/school' 
    #res_path = join(res_path, school_type)
    
    sample_prevention_strategies(params, school, agent_types, res_path, runs,
                                 verbose=False)


IntProgress(value=0, max=260)

## Extract observables

In [5]:
class_sizes = [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
class_numbers = [4, 5, 6, 7, 8, 9, 10, 15]
floor_numbers = [1, 2, 3, 4]
school_types = ['volksschule']

schools = ['{}_classes-{}_students-{}_floors-{}'.format(st, cn, cs, f) for \
          st in school_types for cn in class_numbers for cs in class_sizes for \
          f in floor_numbers]

In [6]:
s_screen_range = [None, 3, 7, 14]
t_screen_range = [None, 3, 7, 14]
half_classes = False
test_types = ['antigen', 'PCR']
index_cases = ['s', 't']
masks = ['T', 'F']
turnovers = [0, 1, 2]
half_classes = ['F']

prevention_strategies = ['test-{}_turnover-{}_index-{}_tf-{}_sf-{}_mask-{}_half-{}'\
        .format(test, turnover, index, tf, sf, mask, half)\
        for test, turnover in zip(test_types, turnovers) \
        for index in ['s', 't'] for tf in t_screen_range \
        for sf in s_screen_range for mask in masks for half in half_classes]

In [8]:
def extract_observables(res_path, schools, prevention_strategies):
    results = pd.DataFrame()
    for i, school in enumerate(schools):
        if i % 10 == 0:
            print('{} / {}'.format(i, len(schools)))
        school_path = join(join(res_path, school.split('_')[0]), school)
        for prev_strategy in prevention_strategies:
            try:
                with open(join(school_path, prev_strategy + '.txt')) as json_file:
                    data = json.load(json_file)
                    row2 = data['observables']
                    for key in ['observables', 'network', 'node_list', 
                                'rep_trans_events', 'schedule']:
                        del data[key]
                    row = data
                    row.update(row2)
            
            except FileNotFoundError:
                row = {col:np.nan for col in results.columns}
            
            results = results.append(row, ignore_index=True)
              
    return results
                

In [10]:
res_path = '../data/school/results'
results = extract_observables(res_path, schools, prevention_strategies)

0 / 352
10 / 352
20 / 352
30 / 352
40 / 352
50 / 352
60 / 352
70 / 352
80 / 352
90 / 352
100 / 352
110 / 352
120 / 352
130 / 352
140 / 352
150 / 352
160 / 352
170 / 352
180 / 352
190 / 352
200 / 352
210 / 352
220 / 352
230 / 352
240 / 352
250 / 352
260 / 352
270 / 352
280 / 352
290 / 352
300 / 352
310 / 352
320 / 352
330 / 352
340 / 352
350 / 352


In [37]:
results['used_tests_median'] = results['N_diagnostic_tests_median'] + results['N_preventive_tests_median']
results['test_per_agent_per_day_median'] = results['used_tests_median'] / results['duration_median'] \
    / (results['classes'] * 2 + results['classes'] * results['students'])

In [38]:
cols = ['school_type', 'classes', 'students', 'floors', 'test_type', \
        'test_turnover', 'screen_frequency_teacher', \
        'screen_frequency_student', 'mask', 'half_classes', 'index_case']

#othercols = [c for c in results.columns if c not in cols and c not in ['student_screen_interval', 'teacher_screen_interval']]
othercols = ['infected_agents_median', 'infected_students_median',
             'infected_teachers_median', 'infected_family_members_median',
            'used_tests_median', 'duration_median', 'quarantine_days_student_median',
            'quarantine_days_teacher_median', 'test_per_agent_per_day_median']

cols.extend(othercols)

In [39]:
results = results[cols]
results.to_csv(join(res_path, 'volksschule.csv'), index=False)

In [58]:
results

,school_type,classes,students,floors,test_type,test_turnover,indexcase,screen_frequency_teacher,screen_frequency_student,mask,...,index_case,infected_agents_median,infected_students_median,infected_teachers_median,used_tests_median,duration_median,quarantine_days_student_median,quarantine_days_teacher_median,predetected_infections_median,undetected_infections_median
0,volksschule,4.0,10.0,1.0,same_day_antigen,0.0,student,None,None,1.0,...,student,1.0,1.0,0.0,0.064516,15.5,7.0,0.0,0.0,0.0
1,volksschule,4.0,10.0,1.0,same_day_antigen,0.0,student,None,None,0.0,...,student,2.0,1.0,0.0,0.050000,20.0,11.0,0.0,0.0,0.0
2,volksschule,4.0,10.0,1.0,same_day_antigen,0.0,student,None,3,1.0,...,student,1.0,1.0,0.0,0.280000,12.5,6.0,0.0,0.0,0.0
3,volksschule,4.0,10.0,1.0,same_day_antigen,0.0,student,None,3,0.0,...,student,2.0,1.0,0.0,0.238506,14.5,7.0,0.0,0.0,0.5
4,volksschule,4.0,10.0,1.0,same_day_antigen,0.0,student,None,7,1.0,...,student,1.0,1.0,0.0,0.095238,14.0,2.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45051,volksschule,15.0,30.0,4.0,one_day_PCR,1.0,teacher,14,3,0.0,...,teacher,5.5,3.5,2.0,0.312571,29.5,44.0,28.0,2.5,1.5
45052,volksschule,15.0,30.0,4.0,one_day_PCR,1.0,teacher,14,7,1.0,...,teacher,11.0,2.5,1.5,0.174823,112.0,265.5,19.5,2.5,0.0
45053,volksschule,15.0,30.0,4.0,one_day_PCR,1.0,teacher,14,7,0.0,...,teacher,18.0,12.0,3.5,0.226849,115.5,268.5,42.0,5.0,1.0
45054,volksschule,15.0,30.0,4.0,one_day_PCR,1.0,teacher,14,14,1.0,...,teacher,9.0,3.0,1.5,0.122936,108.5,200.0,21.0,2.0,2.0
